# Setup command line client

In [1]:
%bookmark root
import os
os.getcwd()

'/home/abr/Downloads/bitmag-release1.9-test'

In [2]:
%cd -b root
%bookmark CLIENT bitrepository-client-1.9-RC1
%env CLIENT bitrepository-client-1.9-RC1
%bookmark -l

(bookmark:root) -> /home/abr/Downloads/bitmag-release1.9-test
/home/abr/Downloads/bitmag-release1.9-test
env: CLIENT=bitrepository-client-1.9-RC1
Current bookmarks:
CLIENT -> bitrepository-client-1.9-RC1
root   -> /home/abr/Downloads/bitmag-release1.9-test


In [3]:
%%bash
pwd
wget -Nq "https://sbforge.org/download/attachments/25395346/${CLIENT}.zip"
rm -rf ${CLIENT}
unzip -q ${CLIENT}.zip

/home/abr/Downloads/bitmag-release1.9-test


In [4]:
%cd -b CLIENT

(bookmark:CLIENT) -> bitrepository-client-1.9-RC1
/home/abr/Downloads/bitmag-release1.9-test/bitrepository-client-1.9-RC1


# Describes the tests needed to validate the GetChecksums functionality.

Upload a file with a known checksum for so we have something to test on

In [6]:
%env TESTFILE1 massupload.sh

env: TESTFILE1=massupload.sh


In [7]:
!bin/bitmag.sh put-file -c integrationtest1 -f bin/$TESTFILE1 -i $TESTFILE1

## Basic getChecksum

In [8]:
!md5sum bin/$TESTFILE1

598c634a8d589b44339cdbffcad1ad44  bin/massupload.sh


### Request the checksum of a file in the collection.

In [9]:
!bin/bitmag.sh get-checksums -c integrationtest1 -i $TESTFILE1

Count: 	Checksum: 	Pillars: 	FileID: 
5 	598c634a8d589b44339cdbffcad1ad44 	All 	massupload.sh


The correct checksum should be returned by all the pillars.

## getChecksum (salted)

### Request the salted checksum of a file in the collection

In [10]:
import hmac
import hashlib
import base64

def getSaltedChecksum(file,saltHex,algorithm):
    #Note, the salt is a hex string
    salt = bytearray.fromhex(saltHex)
    digester = hmac.new(salt,None,algorithm)
    with open(file,'rb') as from_fh:
        while True:
            chunk = from_fh.read()
            if not chunk:
                break
            digester.update(chunk)
    return digester.hexdigest().lower()

file = 'bin/'+os.environ.get('TESTFILE1')
print(getSaltedChecksum(file,saltHex='abcd',algorithm=hashlib.md5))

548de5b292b99df850c65bbfcffe68bc


In [11]:
!bin/bitmag.sh get-checksums -c integrationtest1 -i $TESTFILE -R HMAC_MD5 -S 'abcd'

Count: 	Checksum: 	Pillars: 	FileID: 
4 	548de5b292b99df850c65bbfcffe68bc 	All 	massupload.sh


The correct salted checksum should be returned by all the pillars (except the ChecksumPillar).

## Missing file

Request the checksum of a file that does not exist in the collection.

In [12]:
%%bash
BADFILE=$(echo $TESTFILE1 | rev)
echo "looking for file '$BADFILE'"
bin/bitmag.sh get-checksums -c integrationtest1 -i $BADFILE

looking for file 'hs.daolpussam'
Count: 	Checksum: 	Pillars: 	FileID: 


No results should be returned.